In [11]:
import pyspark
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession

In [12]:
spark = SparkSession.builder.appName("Sales analysis").getOrCreate()

In [13]:
df = spark.read.load('C:\\Users\\pc\\Downloads\\archive\\E-commerce Dataset.csv',format = 'csv',header ='true',inferSchema = 'true')
df.head()

Row(Order_Date=datetime.date(2018, 1, 2), Time=datetime.datetime(2024, 3, 30, 10, 56, 33), Aging=8.0, Customer_Id=37077, Gender='Female', Device_Type='Web', Customer_Login_type='Member', Product_Category='Auto & Accessories', Product='Car Media Players', Sales=140.0, Quantity=1.0, Discount=0.3, Profit=46.0, Shipping_Cost=4.6, Order_Priority='Medium', Payment_method='credit_card')

In [14]:
df.count()

51290

In [15]:
df.printSchema()

root
 |-- Order_Date: date (nullable = true)
 |-- Time: timestamp (nullable = true)
 |-- Aging: double (nullable = true)
 |-- Customer_Id: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Device_Type: string (nullable = true)
 |-- Customer_Login_type: string (nullable = true)
 |-- Product_Category: string (nullable = true)
 |-- Product: string (nullable = true)
 |-- Sales: double (nullable = true)
 |-- Quantity: double (nullable = true)
 |-- Discount: double (nullable = true)
 |-- Profit: double (nullable = true)
 |-- Shipping_Cost: double (nullable = true)
 |-- Order_Priority: string (nullable = true)
 |-- Payment_method: string (nullable = true)



In [16]:
# Creating a temp table for using Sql queries
df.createOrReplaceTempView("sales_data")

In [17]:
spark.sql("SELECT * FROM sales_data").show()

+----------+-------------------+-----+-----------+------+-----------+-------------------+------------------+--------------------+-----+--------+--------+------+-------------+--------------+--------------+
|Order_Date|               Time|Aging|Customer_Id|Gender|Device_Type|Customer_Login_type|  Product_Category|             Product|Sales|Quantity|Discount|Profit|Shipping_Cost|Order_Priority|Payment_method|
+----------+-------------------+-----+-----------+------+-----------+-------------------+------------------+--------------------+-----+--------+--------+------+-------------+--------------+--------------+
|2018-01-02|2024-03-30 10:56:33|  8.0|      37077|Female|        Web|             Member|Auto & Accessories|   Car Media Players|140.0|     1.0|     0.3|  46.0|          4.6|        Medium|   credit_card|
|2018-07-24|2024-03-30 20:41:37|  2.0|      59173|Female|        Web|             Member|Auto & Accessories|        Car Speakers|211.0|     1.0|     0.3| 112.0|         11.2|      

In [18]:
# 1. Customer Segmentation Categorizing customers based on their spendings

customer_data = df.groupBy('Customer_Id').sum('Sales')
#display(customer_data)
customer_data.show()

+-----------+----------+
|Customer_Id|sum(Sales)|
+-----------+----------+
|      37251|     140.0|
|      49855|     249.0|
|      19204|     336.0|
|      45011|     225.0|
|      19553|     283.0|
|      29285|     211.0|
|      37307|     250.0|
|      22346|     575.0|
|      42834|     486.0|
|      33868|     140.0|
|      49308|      54.0|
|      13832|     202.0|
|      48398|     231.0|
|      54264|     250.0|
|      40383|     250.0|
|      38723|     140.0|
|      53691|     680.0|
|      12940|      54.0|
|      32592|      54.0|
|      45307|     548.0|
+-----------+----------+
only showing top 20 rows



In [19]:
from pyspark.sql.functions import when

customer_data = customer_data.withColumn('Segment',when(customer_data['sum(Sales)'] > 500, 'First Class')
.when((customer_data['sum(Sales)'] < 500) & (customer_data['sum(Sales)'] >= 300),'Second Class')
.when(customer_data['sum(Sales)'] < 300,'Third Class')
.otherwise('Unknown'))


In [20]:
display(customer_data.sort(customer_data['sum(Sales)'].desc()))

DataFrame[Customer_Id: int, sum(Sales): double, Segment: string]

In [21]:
# 2. monthly trend of sales  ( how much sales each month ?)

#display(spark.sql('''select month(Order_Date) as Order_month, sum(Sales) as Total_Sales from sales_data
             #group by Order_month 
             #order by Total_Sales desc'''))
spark.sql("select month(Order_Date) as Order_month, sum(Sales) as Total_Sales from sales_data group by Order_month order by Total_Sales desc").show()

+-----------+-----------+
|Order_month|Total_Sales|
+-----------+-----------+
|         11|   877881.0|
|          5|   824502.0|
|          7|   810205.0|
|         12|   767147.0|
|         10|   743387.0|
|          9|   738303.0|
|          8|   664495.0|
|          6|   642555.0|
|          4|   597312.0|
|          3|   435502.0|
|          1|   379627.0|
|          2|   332495.0|
+-----------+-----------+



In [22]:
# 3. Hourly Sales Analysis Which hour has more number of sales? 

display(spark.sql('''select hour(Time) as Sale_Hour, sum(Sales) as Total_Sales from sales_data 
             group by Sale_Hour 
             order by Total_Sales desc'''))


DataFrame[Sale_Hour: int, Total_Sales: double]

In [23]:
# i. Which category product has sold more?
display(spark.sql('''select Product_Category,sum(Sales) as Total_Sales from sales_data
             group by Product_Category
             order by Total_Sales desc'''))

DataFrame[Product_Category: string, Total_Sales: double]

In [24]:
# ii. Which product has sold more?
display(spark.sql('''select Product, count(Product) as number_of_sales from sales_data
             group by Product
             order by number_of_sales desc
             limit 5'''))

DataFrame[Product: string, number_of_sales: bigint]

In [60]:
# i. What are the most commonly used payment types?
display(spark.sql('''select Payment_method, count(Payment_method) as mostly_used_count from sales_data
          group by Payment_method
          order by mostly_used_count desc'''))


DataFrame[Payment_method: string, mostly_used_count: bigint]